In [75]:
import pandas as pd

In [ ]:
#Primero procesar los archivos de energia_activa_2020 hasta 2023
import requests

# URL del archivo en GitHub
urls = [
    'https://github.com/Causil/especializacion/blob/78c82138f4cd58798c69eede6dd5cb6da6a62805/monografia/momentos_evaluativos/Momento%203/datos_originales/Energia%20Activa%202020.xlsx',
    'https://github.com/Causil/especializacion/blob/78c82138f4cd58798c69eede6dd5cb6da6a62805/monografia/momentos_evaluativos/Momento%203/datos_originales/Energia%20activa%202021.xlsx',
    'https://github.com/Causil/especializacion/blob/78c82138f4cd58798c69eede6dd5cb6da6a62805/monografia/momentos_evaluativos/Momento%203/datos_originales/Energia%20activa%202022.xlsx',
    'https://github.com/Causil/especializacion/blob/78c82138f4cd58798c69eede6dd5cb6da6a62805/monografia/momentos_evaluativos/Momento%203/datos_originales/Energ%C3%ADa%20Activa%202023.xlsx'    
    ]

# Realiza la solicitud GET para obtener el contenido del archivo
for url in urls:
    response = requests.get(url)

    # Verifica si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Guarda el contenido del archivo en un archivo local
        with open('Energia_Activa_2020.xlsx', 'wb') as f:
            f.write(response.content)
        print("El archivo se ha descargado exitosamente.")
    else:
        print("Error al descargar el archivo. Código de estado:", response.status_code)

    

In [96]:
#Definiendo rutas de fuentes en bruto
path_data = {
    "auralac_iot_calderas"         :"./Datos_embruto/auralac_iot_calderas.csv",
    "auralac_iot_esterilizadores"  :"./Datos_embruto/auralac_iot_esterilizadores.csv",
    "auralac_iot_pasteurizadores"  :"./Datos_embruto/auralac_iot_pasteurizadores.csv",
    "auralac_iot_power_meters"     :"./Datos_embruto/auralac_iot_power_meters.csv",
    "auralac_iot_silos"            :"./Datos_embruto/auralac_iot_silos.csv",
    "auralac_iot_subscriptions_log":"./Datos_embruto/auralac_iot_subscriptions_log.csv",
    "auralac_iot_yogurteras"       :"./Datos_embruto/auralac_iot_yogurteras.csv",
    "energia_activa_2020"          :"https://github.com/Causil/especializacion/blob/78c82138f4cd58798c69eede6dd5cb6da6a62805/monografia/momentos_evaluativos/Momento%203/datos_originales/Energia%20Activa%202020.xlsx",
    "energia_activa_2021"          :"./Datos_embruto/Energia activa 2021.xlsx",
    "energia_activa_2022"          :"./Datos_embruto/Energia activa 2022.xlsx",
    "energia_activa_2023"          :"./Datos_embruto/Energia activa 2023.xlsx",
}


b'\n\n\n\n\n\n<!DOCTYPE html>\n<html\n  lang="en"\n  \n  data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"\n  data-a11y-animated-images="system" data-a11y-link-underlines="true"\n  >\n\n\n\n\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>\n  <link rel="preconnect" href="https://avatars.githubusercontent.com">\n\n  \n\n\n  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-0f9c7cd68e73.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/dark-13e7ced9cfd1.css" /><link data-color-theme="dark_dimm

In [97]:
df = pd.read_excel(path_data["energia_activa_2020"])
df

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [92]:
#Restringiendo a los datos solo con lo que se necesita


xdf = df.iloc[15:,2:]
xdf.iloc[0,0] = "date"
xdf_sin_nan = xdf.dropna(axis=1)
xdf_sin_nan
#nuevos_nombres_columnas =  list(xdf_sin_nan.iloc[0])
#xdf_sin_nan.columns = nuevos_nombres_columnas
#xdf_sin_nan = xdf_sin_nan.reset_index(drop=True)
#xdf_sin_nan = xdf_sin_nan.drop(0)
#xdf_sin_nan = xdf_sin_nan.reset_index(drop=True)
#xdf_sin_nan

,Unnamed: 2,Unnamed: 3,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 9,Unnamed: 11,Unnamed: 15,Unnamed: 16,Unnamed: 21,...,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108
15,date,Energía Activa Consumo\n(kWh),Energía Activa Generación\n(kWh),Energía Reactiva Inductiva\n(kVarh),Energía Reactiva Capacitiva\n(kVarh),Energía Activa Consumo\n(kWh),Energía Activa Generación\n(kWh),Energía Reactiva Inductiva\n(kVarh),Energía Reactiva Capacitiva\n(kVarh),Energía Activa Consumo\n(kWh),...,Energía Reactiva Inductiva\n(kVarh),Energía Reactiva Capacitiva\n(kVarh),Energía Activa Consumo\n(kWh),Energía Activa Generación\n(kWh),Energía Reactiva Inductiva\n(kVarh),Energía Reactiva Capacitiva\n(kVarh),Energía Activa Consumo\n(kWh),Energía Activa Generación\n(kWh),Energía Reactiva Inductiva\n(kVarh),Energía Reactiva Capacitiva\n(kVarh)
16,16/02/2020,322.32,0,0,0,356.64,0,0,0,375.36,...,158.4,0,362.6,0,179.2,0,377.2,0,178.4,0
17,17/02/2020,380.4,0,178.6,0,402.6,0,191.6,0,410.6,...,295.4,0,533.2,0,280,0,535.2,0,270.6,0
18,18/02/2020,603.2,0,300,0,606,0,292.8,0,616.4,...,356.2,0,604.2,0,296.6,0,567,0,275.8,0
19,19/02/2020,566.8,0,285.4,0,577,0,279.6,0,533,...,313,0,561.6,0,309,0,605.4,0,330.6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,27/12/2020,582.8,0,306.8,0,613,0,311.4,0,671.6,...,323.8,0,609,0,320.8,0,548.4,0,304.2,0
332,28/12/2020,545.8,0,296,0,558.2,0,299.6,0,502.2,...,375,0,720.4,0,396.2,0,706.8,0,387,0
333,29/12/2020,703.4,0,357.8,0,636,0,320.6,0,735.8,...,356,0,645,0,368.8,0,605.6,0,346,0
334,30/12/2020,579,0,336.2,0,600.8,0,322.8,0,585.4,...,317.8,0,623.6,0,317.6,0,663,0,358.8,0
